# Introdução

Para o presente projeto, é necessário fazer o *download* os dados espaciais contendo a divisão municipal. Optou-se por baixar os dados do [**Instituto Brasileiro de Geografia e Estatistica (IBGE)**](https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais), em escala 1:250.000, suficiente para atender o objetivo do projeto, que trata-se de demostrar quais as divisões administrativas, por município, de diversos órgãos e/ou instituições.

Portanto, faz-se o *download* do arquivo zipado do **IBGE** e se descompacta o arquivo _shapefile_. Aproveitou-se para renomear o arquivo um nome mais inteligível.

In [ ]:
import os
import requests
import geopandas as gpd
from zipfile import ZipFile

In [ ]:
# Create Directory
os.makedirs('data/tabs', exist_ok=True)
os.makedirs('data/shps', exist_ok=True)

In [ ]:
# URL para baixar o arquivo zip
url = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2020/UFs/SP/SP_Municipios_2020.zip'

# Define o nome do arquivo zip a ser salvo
name = 'sp_ibge_250k'

# Define o nome do arquivo, com diretório, que será salvo
zipfile = os.path.join('data', 'shps', '{}.zip'.format(name))

# Faz o download do arquivo da 'url' e salva localmente com o nome do arquivo
r = requests.get(url, stream=True)
with open(zipfile, 'wb') as out:
    for chunk in r.iter_content(chunk_size=128):
        out.write(chunk)

In [ ]:
# Get dir of zipfile
path = os.path.dirname(zipfile)

# Create a ZipFile Object
with ZipFile(zipfile, 'r') as zip_obj:
    # Get a list of all archived file names from the zip
    list_files = []
    list_files = zip_obj.namelist()
    
    # Iterate over the file names
    for file in list_files:
        filename, ext = os.path.splitext(file)
        zip_obj.extract(file, os.path.dirname(zipfile))
        os.rename(os.path.join(path, file), os.path.join(path, name+ext))
        print('File "{}" extracted as "{}" in "{}" directory.'.format(file, name+ext, path))

In [ ]:
# Remove file
os.remove(zipfile)

## Converte em *geojson*

In [ ]:
file = 'sp_ibge_250k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)


# Deleta colunas
gdf.drop(['NM_MUN', 'SIGLA_UF', 'AREA_KM2'], axis=1, inplace=True)

# Renomeia colunas
gdf.rename(columns={'CD_MUN': 'id_municipio'}, inplace=True)

# Results
display(gdf.head(5))

In [ ]:
gdf.to_file(
    os.path.join('data', 'shps', '{}.geojson'.format(file)),
    driver='GeoJSON',
    encoding='utf-8'
)

___

# Encerramento

## *Requirements*

In [ ]:
# Input
inp = 'get_spatial_data.ipynb'

# Output
out = os.path.join(inp.split('.')[0])
!jupyter-nbconvert $inp --to python --output $out

print('-'*80)
!pipreqs '.' --force --debug

print('-'*80)
os.remove('{}.py'.format(out))
print('Arquivo {}.py deletado com sucesso!'.format(out))

## *GitHub*

In [ ]:
!nbstripout --install --attributes .gitattributes
!git status
!git add .
!git commit -m 'Arrumações Gerais'
!git push origin main